In [24]:
import polars as pl
import requests

def fetch_news_data(stock, start_date, end_date,turn):
    """
    Makes an API request and returns the response data.
    
    :param endpoint: The API endpoint URL.
    :param headers: Dictionary containing request headers.
    :param params: Dictionary containing query parameters.
    :return: Response JSON as a dictionary or None if the request fails.
    """
    if turn:
        if end_date:
            print("others")
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&time_to={end_date}T0130&limit=1000&apikey=ZRX2N0ETMX8N4GOR"
        else:
            print("65DCZMTJLB2SERT9")
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&limit=1000&apikey=MLPIDKVUZKO71B19"
    else:
        if end_date:
            print("others")
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&time_to={end_date}T0130&limit=1000&apikey=LPHO83KMASND8ZJJ"
        else:
            print("others")
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&limit=1000&apikey=47WWMCYJI8DEB3KR"
    response = requests.get(endpoint)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

def convert_to_dataframe(news_items):
    """
    Converts a list of news items into a Polars DataFrame and extracts the date.
    
    :param news_items: List of news items.
    :return: A Polars DataFrame with an added 'date' column.
    """
    news_items = news_items.get("feed", [])
    if news_items:
        df = pl.DataFrame(news_items)
        df = df.with_columns(pl.col("time_published").str.slice(0, 8).alias("date"))
        return df
    return pl.DataFrame()

def aggregate_news_data(stocks, start_date, end_date, turn):
    """
    Fetches and aggregates news data for a specified number of API calls.
     
    :param num_calls: Number of API calls to make.
    :param endpoint: The API endpoint URL.
    :param headers: Dictionary containing request headers.
    :param params: Dictionary containing query parameters.
    :return: A Polars DataFrame containing aggregated news data.
    """
    df_list = []
    for stock in stocks:
        # Fetch the news data
        news_data = fetch_news_data(stock, start_date, end_date, turn)
        
        if news_data:
            # Convert to DataFrame and append to the list
            df = convert_to_dataframe(news_data)
            if df.height == 0:
                print(news_data)
                print('yes')
                print(df)
            df_list.append(df)

    # Concatenate all DataFrames into a single DataFrame
    return pl.concat(df_list, how='vertical') if df_list else pl.DataFrame()

In [4]:
stocks = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'META', 'BRK-A', 'LLY', 'AVGO']
all_news_df_year_2022 = aggregate_news_data(stocks, 20220101, 20230101, False)

In [7]:
all_news_df_year_2023 = aggregate_news_data(stocks, 20230101, 20240101, False)

In [25]:
all_news_df_year_2024 = aggregate_news_data(stocks, 20240101, False, True)

65DCZMTJLB2SERT9
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
yes
shape: (0, 0)
┌┐
╞╡
└┘
65DCZMTJLB2SERT9
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
yes
shape: (0, 0)
┌┐
╞╡
└┘
65DCZMTJLB2SERT9
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
yes
shape: (0, 0)
┌┐
╞╡
└┘
65DCZMTJLB2SERT9
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans 

In [ ]:
    all_news_df_year_2024 = aggregate_news_data(stocks, 20240101, False, True)
    df = pl.read_json('data_news.json')
    all_news_df = pl.concat([df, all_news_df_year_2024])#, all_news_df_year_2024])
    # Convert the DataFrame to a JSON string
    json_str = all_news_df.write_json()
    #print(json_str)
    # Specify the file path
    file_path = 'data_news2.json'

    # Write the JSON string to a file
    with open(file_path, 'w') as f:
        f.write(json_str)

    # Confirm the JSON data has been saved
    print(f'JSON data saved to {file_path}')


if __name__ == "__main__":
    main()


In [12]:
all_news_df_year_2023

title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,date
str,str,str,list[str],str,str,str,str,str,list[struct[2]],f64,str,list[struct[4]],str
"""What to stream…","""https://www.ma…","""20231231T21160…","[""Mike Murphy""]","""As the calenda…","""https://images…","""MarketWatch""","""Top Stories""","""www.marketwatc…","[{""Technology"",""0.5""}, {""Retail & Wholesale"",""0.5""}]",0.056158,"""Neutral""","[{""NFLX"",""0.108141"",""0.056923"",""Neutral""}, {""WBD"",""0.072217"",""0.058011"",""Neutral""}, … {""AMZN"",""0.072217"",""-0.027629"",""Neutral""}]","""20231231"""
"""3 Warren Buffe…","""https://www.fo…","""20231231T15000…","[""Adam Levy""]","""These three st…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""Financial Markets"",""0.962106""}, {""Earnings"",""0.947132""}, … {""Finance"",""0.5""}]",0.366263,"""Bullish""","[{""TMUS"",""0.081276"",""0.096784"",""Neutral""}, {""AAPL"",""0.240489"",""0.225259"",""Somewhat-Bullish""}, … {""BRK-A"",""0.040691"",""0.100149"",""Neutral""}]","""20231231"""
"""Almost Half of…","""https://www.fo…","""20231231T14530…","[""Neil Patel""]","""This well-know…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""Financial Markets"",""0.413559""}, {""Manufacturing"",""0.333333""}, … {""Finance"",""0.333333""}]",0.231095,"""Somewhat-Bulli…","[{""AXP"",""0.06697"",""0.18865"",""Somewhat-Bullish""}, {""AAPL"",""0.725359"",""0.424969"",""Bullish""}, … {""BRK-A"",""0.13347"",""0.188494"",""Somewhat-Bullish""}]","""20231231"""
"""Why Artificial…","""https://www.fo…","""20231231T12300…","[""Harsh Chauhan""]","""This tech tita…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""Economy - Monetary"",""0.158519""}, {""Financial Markets"",""0.316726""}, … {""Technology"",""0.5""}]",0.236511,"""Somewhat-Bulli…","[{""MSFT"",""0.045224"",""0.160039"",""Somewhat-Bullish""}, {""META"",""0.045224"",""0.160039"",""Somewhat-Bullish""}, … {""TSLA"",""0.045224"",""0.160039"",""Somewhat-Bullish""}]","""20231231"""
"""$10,000 Invest…","""https://www.fo…","""20231231T10300…","[""RJ Fulton""]","""As the cryptoc…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""Retail & Wholesale"",""0.333333""}, {""Financial Markets"",""0.360215""}, … {""Blockchain"",""0.769861""}]",0.186947,"""Somewhat-Bulli…","[{""AAPL"",""0.053509"",""0.186986"",""Somewhat-Bullish""}, {""AMZN"",""0.053509"",""0.186986"",""Somewhat-Bullish""}, … {""CRYPTO:BTC"",""0.053509"",""0.1067"",""Neutral""}]","""20231231"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Broadcom Stock…","""https://www.be…","""20230821T15174…","[""Adam Eckert""]","""Broadcom Inc A…","""https://cdn.be…","""Benzinga""","""Mergers""","""www.benzinga.c…","[{""Technology"",""0.5""}, {""Financial Markets"",""0.108179""}, {""Manufacturing"",""0.5""}]",0.339224,"""Somewhat-Bulli…","[{""VMW"",""0.640349"",""0.56118"",""Bullish""}, {""AVGO"",""0.960704"",""0.65282"",""Bullish""}]","""20230821"""
"""Stock Market M…","""https://www.in…","""20230821T14500…","[""Investor's Business Daily"", ""KIMBERLEY KOENIG""]","""Stock Market M…","""https://www.in…","""Investors Busi…","""n/a""","""www.investors.…","[{""Energy & Transportation"",""0.25""}, {""Technology"",""0.25""}, … {""Mergers & Acquisitions"",""0.360215""}]",0.202316,"""Somewhat-Bulli…","[{""XPEV"",""0.12645"",""0.265697"",""Somewhat-Bullish""}, {""NSSC"",""0.12645"",""-0.030647"",""Neutral""}, … {""PANW"",""0.12645"",""0.061335"",""Neutral""}]","""20230821"""
"""Why These 2 Te…","""https://www.fo…","""20230821T13552…","[""Dan Caplinger""]","""Wall Street go…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""Financial Markets"",""0.77141""}, {""Manufacturing"",""0.5""}, … {""Technology"",""0.5""}]",0.333837,"""Somewhat-Bulli…","[{""VMW"",""0.293741"",""0.356722"",""Bullish""}, {""AVGO"",""0.293741"",""0.294625"",""Somewhat-

In [26]:
all_news_df = pl.concat([all_news_df_year_2022, all_news_df_year_2023])#, all_news_df_year_2024])

In [27]:
all_news_df.shape

(13888, 14)

In [28]:
all_news_df.write_csv("2022_2023_data.csv")

ComputeError: CSV format does not support nested data

In [33]:
json_str = all_news_df.write_json()
#print(json_str)
# Specify the file path
file_path = '2022_2023_data.json'

# Write the JSON string to a file
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(json_str)

In [30]:
all_news_df

title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,date
str,str,str,list[str],str,str,str,str,str,list[struct[2]],f64,str,list[struct[4]],str
"""Apple, AMC, Me…","""https://www.be…","""20221231T19031…","[""Michael Cohen""]","""Benzinga exami…","""https://cdn.be…","""Benzinga""","""Trading""","""www.benzinga.c…","[{""Financial Markets"",""0.316726""}, {""Manufacturing"",""0.333333""}, … {""Technology"",""0.333333""}]",-0.079733,"""Neutral""","[{""MSTR"",""0.247838"",""0.0"",""Neutral""}, {""META"",""0.166743"",""-0.022036"",""Neutral""}, … {""CRYPTO:BTC"",""0.326283"",""0.0"",""Neutral""}]","""20221231"""
"""Bulls In A Bea…","""https://www.be…","""20221231T16111…","[""Shanthi Rexaline""]","""2022 would go …","""https://cdn.be…","""Benzinga""","""General""","""www.benzinga.c…","[{""Life Sciences"",""0.166667""}, {""Energy & Transportation"",""0.166667""}, … {""Real Estate & Construction"",""0.166667""}]",0.064077,"""Neutral""","[{""AAPL"",""0.113776"",""-0.055435"",""Neutral""}, {""TSLA"",""0.075995"",""-0.147409"",""Neutral""}, … {""GS"",""0.03804"",""-0.034304"",""Neutral""}]","""20221231"""
"""Where Will Uni…","""https://www.fo…","""20221231T15450…","[""Leo Sun""]","""The game-engin…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""IPO"",""0.769861""}, {""Financial Markets"",""0.266143""}, … {""Technology"",""1.0""}]",0.202065,"""Somewhat-Bulli…","[{""U"",""0.095517"",""0.0"",""Neutral""}, {""AAPL"",""0.095517"",""0.133304"",""Neutral""}]","""20221231"""
"""3 Unstoppable …","""https://www.fo…","""20221231T14000…","[""Dani Cook""]","""These companie…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""Financial Markets"",""0.999864""}, {""Manufacturing"",""0.5""}, … {""Technology"",""0.5""}]",0.152163,"""Somewhat-Bulli…","[{""AMD"",""0.223248"",""0.034875"",""Neutral""}, {""MSFT"",""0.349945"",""0.119746"",""Neutral""}, {""AAPL"",""0.308614"",""-0.039157"",""Neutral""}]","""20221231"""
"""Even in an Adv…","""https://www.fo…","""20221231T12000…","[""Adam Levy""]","""Ad sales growt…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""Earnings"",""0.986714""}, {""Technology"",""1.0""}, … {""Economy - Macro"",""0.158519""}]",0.151938,"""Somewhat-Bulli…","[{""GOOG"",""0.243376"",""0.076203"",""Neutral""}, {""ROKU"",""0.049424"",""-0.017034"",""Neutral""}, … {""TTD"",""0.335628"",""0.009787"",""Neutral""}]","""20221231"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Broadcom Stock…","""https://www.be…","""20230821T15174…","[""Adam Eckert""]","""Broadcom Inc A…","""https://cdn.be…","""Benzinga""","""Mergers""","""www.benzinga.c…","[{""Technology"",""0.5""}, {""Financial Markets"",""0.108179""}, {""Manufacturing"",""0.5""}]",0.339224,"""Somewhat-Bulli…","[{""VMW"",""0.640349"",""0.56118"",""Bullish""}, {""AVGO"",""0.960704"",""0.65282"",""Bullish""}]","""20230821"""
"""Stock Market M…","""https://www.in…","""20230821T14500…","[""Investor's Business Daily"", ""KIMBERLEY KOENIG""]","""Stock Market M…","""https://www.in…","""Investors Busi…","""n/a""","""www.investors.…","[{""Energy & Transportation"",""0.25""}, {""Technology"",""0.25""}, … {""Mergers & Acquisitions"",""0.360215""}]",0.202316,"""Somewhat-Bulli…","[{""XPEV"",""0.12645"",""0.265697"",""Somewhat-Bullish""}, {""NSSC"",""0.12645"",""-0.030647"",""Neutral""}, … {""PANW"",""0.12645"",""0.061335"",""Neutral""}]","""20230821"""
"""Why These 2 Te…","""https://www.fo…","""20230821T13552…","[""Dan Caplinger""]","""Wall Street go…","""https://g.fool…","""Motley Fool""","""n/a""","""www.fool.com""","[{""Financial Markets"",""0.77141""}, {""Manufacturing"",""0.5""}, … {""Technology"",""0.5""}]",0.333837,"""Somewhat-Bulli…","[{""VMW"",""0.293741"",""0.356722"",""Bullish""}, {""AVGO"",""0.293741"",""0.294625"",""Somewhat-Bullish""}, {""PANW"",""0.1789"",""0.229178"",""Somewhat-Bullish""}]","""20230821"""


In [34]:
import json

In [43]:
import pandas as pd

# Read the JSON data
df = pd.read_json('2022_2023_data.json')

In [52]:
df

,columns
0,"{'name': 'title', 'datatype': 'String', 'bit_s..."
1,"{'name': 'url', 'datatype': 'String', 'bit_set..."
2,"{'name': 'time_published', 'datatype': 'String..."
3,"{'name': 'authors', 'datatype': {'List': 'Stri..."
4,"{'name': 'summary', 'datatype': 'String', 'bit..."
5,"{'name': 'banner_image', 'datatype': 'String',..."
6,"{'name': 'source', 'datatype': 'String', 'bit_..."
7,"{'name': 'category_within_source', 'datatype':..."
8,"{'name': 'source_domain', 'datatype': 'String'..."
9,"{'name': 'topics', 'datatype': {'List': {'Stru..."


In [53]:
df = all_news_df.to_pandas()

In [55]:
df['date'].nunique()

310

In [56]:
df.to_csv('2022_2023_data.csv', index = False)